In [1]:
# visualize the boxplot of the training experiment
import os.path as osp
import os
import numpy as np
import pandas as pd
import json
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
# 1. 5 models fine-tuned or trained on STF dataset
# 2. 5 models trained on diff dataset without fine-tuning
# 3. all models in comparisons

In [6]:
# specify the model paths
metrics_root = "/labs/gevaertlab/users/yyhhli/code/vae/EDA/training_exp_eval/results"
prefix = "TRAINING_EXPS_V2-"
# fine-tuned or trained on STF dataset
model_paths_v1 = [
    'lidc->stf_4e-6_step5000_version-4.json',
    'lndb->stf_4e-6_step5000_version-1.json',
    'lidc-lndb->stf_4e-6_step5000_version-2.json',
    'stf-aug_8-32-128-512_1e-5_step65000_version-2.json',
    ]
# trained on diff dataset without fine-tuning
model_paths_v2 = [
    'lidcaug_8-32-128-512_1e-5_step65000_version-3.json',
    'lndb->stf_4e-6_step5000_version-1.json',
    'lidc-lndb-aug_8-32-128-512_1e-5_step65000_version-2.json',
    'lidc-lndb-stf-aug_8-32-128-512_1e-5_step65000_version-1.json',
    'stf-aug_8-32-128-512_1e-5_step65000_version-2.json',
    ]
# all models in comparisons
# model_paths_v3 = [
#     'TRAINING_EXPS_V2-lndb->stf_4e-6_step5000_version-1.json',
#     'TRAINING_EXPS_V2-lidcaug_8-32-128-512_1e-5_step65000_version-3.json',
#     'TRAINING_EXPS_V2-lidc-lndb-aug_8-32-128-512_1e-5_step65000_version-2.json',
#     'TRAINING_EXPS_V2-stf-aug_8-32-128-512_1e-5_step65000_version-2.json',
#     'TRAINING_EXPS_V2-lidc-lndb-stf-aug_8-32-128-512_1e-5_step65000_version-1.json',
#     'TRAINING_EXPS_V2-lndb-aug_8-32-128-512_1e-5_step65000_version-1.json',
#     'TRAINING_EXPS_V2-lidc->stf_4e-6_step5000_version-4.json',
#     'TRAINING_EXPS_V2-lidc-lndb->stf_4e-6_step5000_version-2.json',
#     'VAE3D32AUG_version-70.json'
#     ]

In [8]:
# draw the boxplot or violin plot

['TRAINING_EXPS_V2-lndb->stf_4e-6_step5000_version-1.json',
 'TRAINING_EXPS_V2-lidcaug_8-32-128-512_1e-5_step65000_version-3.json',
 'TRAINING_EXPS_V2-lidc-lndb-aug_8-32-128-512_1e-5_step65000_version-2.json',
 'TRAINING_EXPS_V2-stf-aug_8-32-128-512_1e-5_step65000_version-2.json',
 'TRAINING_EXPS_V2-lidc-lndb-stf-aug_8-32-128-512_1e-5_step65000_version-1.json',
 'TRAINING_EXPS_V2-lndb-aug_8-32-128-512_1e-5_step65000_version-1.json',
 'TRAINING_EXPS_V2-lidc->stf_4e-6_step5000_version-4.json',
 'TRAINING_EXPS_V2-lidc-lndb->stf_4e-6_step5000_version-2.json',
 'VAE3D32AUG_version-70.json']